In [1]:
import numpy   as np
import pandas  as pd
import seaborn as sns

import df_visualizations as dv
import remap_values as rv

import matplotlib.pyplot as plt

from sklearn.decomposition import PCA

In [2]:
%matplotlib inline
plt.style.use( 'dark_background' )
plt.rcParams['figure.figsize'] = (14,10)

In [6]:
use_list = [
    'logerror','transactiondate','airconditioningtypeid','pooltypeid10', # Hotub
    'poolsizesum','garagetotalsqft','calculatedbathnbr','fireplacecnt',
    'finishedsquarefeet12','lotsizesquarefeet','taxdelinquencyflag','yearbuilt',
    'structuretaxvaluedollarcnt','taxvaluedollarcnt','landtaxvaluedollarcnt','taxamount',
    'decktypeid','numberofstories','buildingqualitytypeid','unitcnt','roomcnt' ]

In [7]:
prop  = pd.read_csv( 'data/properties_2016.csv' )
trans = pd.read_csv( 'data/train_2016_v2.csv'   )

my_df = pd.merge( prop, trans, on='parcelid', how='inner' )
my_df = my_df[ use_list ]

del prop
del trans

my_df['transactiondate' ] =   pd.to_datetime  ( my_df['transactiondate'] )
my_df['time_since_built'] = ( pd.DatetimeIndex( my_df['transactiondate'] ).year - my_df['yearbuilt'] )
my_df['ordinal_date'    ] =  [x.toordinal() for x in my_df['transactiondate']]
my_df['day_of_week'     ] = ( my_df['ordinal_date'] - my_df['ordinal_date'].min() + 2 ) % 7

my_df = my_df.drop( ['yearbuilt'], axis=1 )

print my_df.shape

(90275, 23)


In [8]:
my_df.columns.values

array(['logerror', 'transactiondate', 'airconditioningtypeid',
       'pooltypeid10', 'poolsizesum', 'garagetotalsqft',
       'calculatedbathnbr', 'fireplacecnt', 'finishedsquarefeet12',
       'lotsizesquarefeet', 'taxdelinquencyflag',
       'structuretaxvaluedollarcnt', 'taxvaluedollarcnt',
       'landtaxvaluedollarcnt', 'taxamount', 'decktypeid',
       'numberofstories', 'buildingqualitytypeid', 'unitcnt', 'roomcnt',
       'time_since_built', 'ordinal_date', 'day_of_week'], dtype=object)

Lets decide what we are doing with the AC

1 Central 2 Chilled Water 3 Evaporative Cooler 4 Geo Thermal 5 None 6 Other 7 Packaged AC Unit 8 Partial 9 Refrigeration 10 Ventilation 11 Wall Unit 12 Window Unit 13 Yes

In [49]:
my_df['has_ac'] = 1

my_df.ix[ my_df['airconditioningtypeid'].isnull(), ['has_ac'] ] = 0
my_df.ix[ my_df['airconditioningtypeid'] == 5    , ['has_ac'] ] = 0

my_df = my_df.drop( 'airconditioningtypeid', axis=1 )

Now it's only a binary flag

Now do the pool, flags for if present, square footage for pool

In [59]:
my_df['has_spa'  ] = 0
my_df['has_pool' ] = 0
my_df['pool_sqft'] = 0

my_df['has_spa'  ] = my_df['pooltypeid10'].fillna(0).astype(int)
my_df['pool_sqft'] = my_df['poolsizesum' ]
my_df['has_pool' ] =(my_df['poolsizesum' ] > 0).astype(int)

my_df = my_df.drop( ['pooltypeid10','poolsizesum'], axis=1 )

Two flags for has pool and has spa, and pool square footage

In [60]:
my_df['unit_sqft'] = my_df['finishedsquarefeet12']
my_df[ 'lot_sqft'] = my_df['lotsizesquarefeet'   ]

my_df = my_df.drop( ['finishedsquarefeet12', 'lotsizesquarefeet'], axis=1 )

Size of the lots

In [62]:
my_df['tax_delinquent'] = my_df['taxdelinquencyflag'].fillna(0).replace( {'Y', 1} )

my_df = my_df.drop( 'taxdelinquencyflag', axis=1 )

Flag for tax delinquency didn't need changing

Building quality broken into categories, low value is higher quality

In [70]:
my_df['building_quality_low' ] = 0
my_df['building_quality_med' ] = 0
my_df['building_quality_high'] = 0
my_df['building_quality_unkn'] = 0

my_df.ix[  my_df['buildingqualitytypeid'] < 4     , 'building_quality_high' ] = 1
my_df.ix[  my_df['buildingqualitytypeid'] > 8     , 'building_quality_low'  ] = 1
my_df.ix[ (my_df['buildingqualitytypeid'] > 3)    &
       (   my_df['buildingqualitytypeid'] < 9)    , 'building_quality_med'  ] = 1
my_df.ix[  my_df['buildingqualitytypeid'].isnull(), 'building_quality_unkn' ] = 1

my_df = my_df.drop( 'buildingqualitytypeid', axis=1 )

KeyError: 'buildingqualitytypeid'

In [71]:
my_df.columns.values

array(['logerror', 'transactiondate', 'garagetotalsqft',
       'calculatedbathnbr', 'fireplacecnt', 'structuretaxvaluedollarcnt',
       'taxvaluedollarcnt', 'landtaxvaluedollarcnt', 'taxamount',
       'decktypeid', 'numberofstories', 'unitcnt', 'roomcnt',
       'time_since_built', 'ordinal_date', 'day_of_week', 'has_ac',
       'has_spa', 'has_pool', 'pool_sqft', 'unit_sqft', 'lot_sqft',
       'tax_delinquent', 'building_quality_low', 'building_quality_med',
       'building_quality_high', 'building_quality_unkn'], dtype=object)

In [75]:
my_df['has_garage' ] = 0
my_df['garage_sqft'] = my_df['garagetotalsqft'].fillna(0)

my_df.ix[ my_df['garagetotalsqft']>1, 'has_garage' ] = 1

my_df = my_df.drop( 'garagetotalsqft', axis=1 )

Garage compressed to sqft, and flag for a garage

In [80]:
my_df['n_fireplaces'] = my_df['fireplacecnt'].fillna(0)
my_df['has_deck'    ] = my_df['decktypeid'  ].notnull().astype(int)

my_df = my_df.drop( ['fireplacecnt','decktypeid'], axis=1 )

1 hot encoding